# SharePoint und Graph API Authentifizierungsbeispiele in Python

In diesem Beitrag zeigen wir **Python-Codebeispiele** für verschiedene Authentifizierungsmethoden, um Daten aus einer SharePoint-Liste auszulesen. Wir betrachten sowohl die **SharePoint REST API** (direkter Aufruf der SharePoint-Endpoints) als auch die **Microsoft Graph API** (moderne API für Office 365 Dienste einschließlich SharePoint). Alle Beispiele lesen ihre Zugangsdaten aus einer zentralen Konfigurationsdatei und schreiben die abgerufenen Listeneinträge in einen Pandas **DataFrame**. Dabei behandeln wir sowohl Legacy-Authentifizierungsmethoden (ältere Verfahren, teils nicht mehr unterstützt) als auch moderne Authentifizierungsverfahren. Paging wird in allen Beispielen berücksichtigt, sodass _alle_ Listeneinträge (alle Felder) geladen werden, auch wenn die Liste sehr groß ist.  

  

**Hinweis:** Einige der gezeigten Legacy-Methoden funktionieren in der Praxis nicht mehr in SharePoint Online bzw. mit der Graph API, was im Text jeweils erläutert wird. Moderne Cloud-Umgebungen erfordern in der Regel **OAuth-basierte** Verfahren (Azure AD Tokens). Legacy-Authentifizierungen wie **WS-Trust**, **Forms Based Auth (FBA)** oder **Basic Auth** sind entweder unsicher oder wurden bereits deaktiviert \[[learn.microsoft.com](https://learn.microsoft.com/en-us/sharepoint/technical-reference/basic-auth-is-being-deprecated#:~:text=Summary%3A%20Basic%20authentication%20is%20currently,from%20SharePoint%20Server%20Subscription%20Edition)\]. Diese zeigen wir der Vollständigkeit halber und um zu erklären, warum sie nicht mehr empfohlen sind.

## Inhaltsverzeichnis

0 | Vorbereitung: Konfiguration mit Zugangsdaten  
1 | SharePoint REST API  
1.1 | Legacy-Authentifizierung (SharePoint REST)  
1.1.1 | WS-Trust (Active Authentication mit SAML Token)  
1.1.2 | Forms-Based Authentication (FBA)  
1.1.3 | Basic Authentication  
1.2 | Moderne Authentifizierung (SharePoint REST)  
1.2.1 | Delegierte Berechtigungen  
1.2.1.1 | Device Code Flow (MSAL)  
1.2.2 | App-Only (Client Credentials)  
1.2.2.1 | Mit Client Secret  
1.2.2.2 | Mit Zertifikat

2 | Microsoft Graph API  
2.1 | Legacy-Authentifizierung (Graph; in der Praxis nicht nutzbar)  
2.1.1 | WS-Trust  
2.1.2 | Forms-Based Authentication (FBA)  
2.1.3 | Basic Authentication  
2.2 | Moderne Authentifizierung (Graph)  
2.2.1 | Delegierte Berechtigungen  
2.2.1.1 | Device Code Flow (MSAL, z. B. Scope `Sites.Read.All`)  
2.2.2 | App-Only (Client Credentials)  
2.2.2.1 | Mit Client Secret  
2.2.2.2 | Mit Zertifikat

3 | Gemeinsame Aspekte  
3.1 | Paging (`__next` bei SharePoint REST, `@odata.nextLink` bei Graph)  
3.2 | Alle Felder laden (keine `$select`\-Einschränkung)  
3.3 | Output in Pandas DataFrame  
3.4 | Fehlerbilder & Troubleshooting (401/403, Audience/Scopes, Consent, Sites.Selected)  
3.5 | Sicherheit & Least Privilege (Admin Consent, Sites.Selected, Secrets vs. Zertifikate)

4 | Zusammenfassung & Best Practices

# 0 | Vorbereitung: Konfigurationsdatei mit Zugangsdaten

Alle Scripts nutzen eine JSON-Konfigurationsdatei (z.B. `config.json`), um sensible Daten nicht im Code zu hinterlegen. Die Datei hat etwa folgenden Aufbau:

In [ ]:
{
  "azuread": {
    "tenant_id": "<Ihr Azure AD Tenant ID>",
    "client_id": "<Ihre Azure AD App Client ID>",
    "client_secret": "<Ihr Azure AD App Client Secret>",
    "cert_path": "<Pfad zu Ihrem Zertifikat.pem (falls benötigt)>",
    "cert_thumbprint": "<Thumbprint des Zertifikats (falls benötigt)>"
  },
  "sharepoint": {
    "username": "<Ihr SharePoint-Benutzername>",
    "password": "<Ihr SharePoint-Passwort>"
  }
}

Jedes Skript lädt diese Konfiguration und extrahiert die benötigten Werte:

In [ ]:
import json
config_path = r"C:\python\Scripts\config.json"
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)

TENANT_ID     = config["azuread"]["tenant_id"]
CLIENT_ID     = config["azuread"]["client_id"]
CLIENT_SECRET = config["azuread"]["client_secret"]

SHAREPOINT_USERNAME = config["sharepoint"]["username"]
SHAREPOINT_PASSWORD = config["sharepoint"]["password"]

Zusätzlich definieren wir zwei Variablen für die **Ziel-Website** und die **Zielliste** in SharePoint, die in allen Beispielen gleich bleiben. Ersetzen Sie diese durch Ihre tatsächliche Site-URL und den Listennamen:

In [ ]:
SITE_URL  = "https://<IhreDomain>.sharepoint.com/sites/<IhrSeitenname>"
LIST_NAME = "<Name Ihrer Liste>"

Im Folgenden gliedern wir die Beispiele nach API (SharePoint vs. Graph) und Authentifizierungsart.

# 1 | SharePoint REST API

Für die SharePoint REST API nutzen wir direkte HTTP-Requests an `SITE_URL/_api/...` Endpunkte. Wir zeigen zunächst 

- Legacy-Authentifizierungsmethoden (1.1.x)
- danach moderne OAuth-Verfahren (1.2.x). 

<span style="color: var(--vscode-foreground);">Ziel ist es stets, die Listeneinträge von </span> `LIST_NAME` <span style="color: var(--vscode-foreground);"> auszulesen.</span>

## Exkurs: Arten von Sharepoint APIs

## 1.1 | Legacy-Authentifizierung (SharePoint REST)

Legacy-Authentifizierungen stammen aus früheren SharePoint-Versionen oder -Architekturen (vor OAuth). Diese Verfahren sind für SharePoint **Online** heute größtenteils nicht mehr zugelassen oder sinnvoll. Im SharePoint-On-Premises-Umfeld können sie teils noch funktionieren, werden aber aus Sicherheitsgründen abgelöst \[[learn.microsoft.com](https:\learn.microsoft.com\en-us\sharepoint\technical-reference\basic-auth-is-being-deprecated#:~:text=Basic%20authentication%20doesn%27t%20provide%20confidentiality,as%20soon%20as%20possible)\]. Wir stellen sie hier vor und weisen auf die Einschränkungen hin.

### 1.1.1 | WS-Trust (Active Authentication mit SAML Token)

**WS-Trust** ist ein Protokoll, mit dem sich ein Client per Benutzername/Passwort am **Security Token Service (STS)** anmeldet und ein SAML-Sicherheitstoken erhält. SharePoint Online nutzte früher diesen Weg in Kombination mit der Microsoft Online STS (`extSTS.srf`), um einen FedAuth-Cookie für SharePoint zu erhalten \[[blog.josephvelliah.com](https:\blog.josephvelliah.com\access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=Generate%20Security%20Token)[blog.josephvelliah.com](https:\blog.josephvelliah.com\access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=)\]. Dieses Verfahren wird als **"Active Authentication"** bezeichnet – d.h. ohne Browser, direkt per Webservice \[[sharepoint.stackexchange.com](https:\sharepoint.stackexchange.com\questions\139209\sharepoint-rest-api-authentication-with-saml#:~:text=SharePoint%20Online%20uses%20a%20token,security%20token%20from%20the%20cookies)\].

**Ablauf:** Der Client sendet eine SOAP-Anfrage an `login.microsoftonline.com/extSTS.srf` mit Benutzer und Passwort, erhält ein SAML-Token zurück, meldet sich mit diesem Token am SharePoint-URL (`/_forms/default.aspx?wa=wsignin1.0`) an, wodurch Auth-Cookies (FedAuth, rtFA) gesetzt werden \[[blog.josephvelliah.com](https:\blog.josephvelliah.com\access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=Now%20hit%20Send%20button%20to,should%20be%20something%20like%20this)[blog.josephvelliah.com](https:\blog.josephvelliah.com\access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=The%20response%20for%20this%20request,down%20the%20values%20of%20the%C2%A0rtFa%C2%A0and%C2%A0FedAuth%C2%A0Cookies)\]. Anschließend können Requests mit diesen Cookies an die REST-API gestellt werden.

**Einschränkung:** Microsoft hat WS-Trust für SharePoint Online weitgehend abgekündigt. In modernen Tenants ist eine direkte WS-Trust-Anmeldung mit Benutzer/Passwort oft nicht mehr möglich (insb. bei MFA, oder wenn der Tenant auf **Modern Authentication only** steht). In solchen Fällen muss auf OAuth (Azure AD) ausgewichen werden. Dieses Beispiel dient vor allem der Illustration; in der Praxis sollte es durch moderne Methoden ersetzt werden.

In [ ]:
# WS-Trust Authentifizierung und Listenzugriff:

import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Konfiguration und Ziel festlegen
config_path = r"C:\python\Scripts\config.json"
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)

USERNAME = config["sharepoint"]["username"]
PASSWORD = config["sharepoint"]["password"]

SITE_URL  = "https://<IhreDomain>.sharepoint.com/sites/<IhrSeitenname>"
LIST_NAME = "<Name Ihrer Liste>"

# 1. SOAP-Request an STS senden, um SAML-Security Token zu erhalten
sts_url = "https://login.microsoftonline.com/extSTS.srf"
login_url = f"{SITE_URL}/_forms/default.aspx?wa=wsignin1.0"
soap_body = f"""
<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope"
            xmlns:a="http://www.w3.org/2005/08/addressing"
            xmlns:o="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-secext-1.0.xsd"
            xmlns:t="http://schemas.xmlsoap.org/ws/2005/02/trust"
            xmlns:wsp="http://schemas.xmlsoap.org/ws/2004/09/policy">
  <s:Header>
    <a:Action s:mustUnderstand="1">http://schemas.xmlsoap.org/ws/2005/02/trust/RST/Issue</a:Action>
    <a:ReplyTo>
      <a:Address>http://www.w3.org/2005/08/addressing/anonymous</a:Address>
    </a:ReplyTo>
    <a:To s:mustUnderstand="1">{sts_url}</a:To>
    <o:Security s:mustUnderstand="1">
      <o:UsernameToken>
        <o:Username>{USERNAME}</o:Username>
        <o:Password>{PASSWORD}</o:Password>
      </o:UsernameToken>
    </o:Security>
  </s:Header>
  <s:Body>
    <t:RequestSecurityToken>
      <wsp:AppliesTo>
        <a:EndpointReference>
          <a:Address>{login_url}</a:Address>
        </a:EndpointReference>
      </wsp:AppliesTo>
      <t:KeyType>http://schemas.xmlsoap.org/ws/2005/05/identity/NoProofKey</t:KeyType>
      <t:RequestType>http://schemas.xmlsoap.org/ws/2005/02/trust/Issue</t:RequestType>
      <t:TokenType>urn:oasis:names:tc:SAML:1.0:assertion</t:TokenType>
    </t:RequestSecurityToken>
  </s:Body>
</s:Envelope>"""
headers = {"Content-Type": "application/soap+xml; charset=utf-8"}

sts_response = requests.post(sts_url, data=soap_body, headers=headers)
# SAML-Token aus der Response extrahieren (BinarySecurityToken)
root = ET.fromstring(sts_response.text)
token_element = root.find('.//wsse:BinarySecurityToken', namespaces={
    'wsse': 'http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-utility-1.0.xsd'
})
if token_element is None:
    raise Exception("Kein Security Token erhalten (WS-Trust fehlgeschlagen).")
saml_token = token_element.text

# 2. Mit dem SAML Token beim SharePoint einloggen (FedAuth Cookie erhalten)
auth_response = requests.post(login_url, data=saml_token,
                              headers={"Content-Type": "application/x-www-form-urlencoded"})
if auth_response.status_code != 200:
    raise Exception("Anmeldung mit SAML-Token fehlgeschlagen.")
cookies = auth_response.cookies  # enthält FedAuth und rtFA Cookies

# 3. REST-Aufruf an SharePoint (Listeneinträge abrufen, mit Paging)
list_url = f"{SITE_URL}/_api/web/Lists/GetByTitle('{LIST_NAME}')/Items"
all_items = []
while True:
    resp = requests.get(list_url, cookies=cookies, headers={"Accept": "application/json"})
    data = resp.json()
    # Ergebnisse sammeln
    items = data.get('d', {}).get('results', [])
    all_items.extend(items)
    # Prüfen, ob es eine nächste Seite gibt
    next_url = data.get('d', {}).get('__next')
    if next_url:
        list_url = next_url  # nächste Page abrufen
    else:
        break

df = pd.DataFrame(all_items)
print(f"{len(df)} Einträge geladen")
print(df.head())

**Erläuterung:** Zunächst wird eine SOAP-Nachricht an `extSTS.srf` geschickt, um ein SAML **BinarySecurityToken** im Response zu erhalten. Dieses wird dann per POST an `/_forms/default.aspx?wa=wsignin1.0` gesendet, was – sofern erfolgreich – Session-Cookies (`FedAuth`, `rtFA`) für die Authentifizierung setzt \[[blog.josephvelliah.com](https:\blog.josephvelliah.com\access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=The%20response%20for%20this%20request,down%20the%20values%20of%20the%C2%A0rtFa%C2%A0and%C2%A0FedAuth%C2%A0Cookies)[blog.josephvelliah.com](https:\blog.josephvelliah.com\access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=The%20response%20for%20this%20request,down%20the%20values%20of%20the%C2%A0rtFa%C2%A0and%C2%A0FedAuth%C2%A0Cookies)\]. Mit diesen Cookies können wir anschließend HTTP-GET Requests an `/_api/web/...` Endpoints ausführen. Im Code wird auf `.../Lists/GetByTitle('ListName')/Items` zugegriffen. Die Ergebnisse kommen seitenweise (SharePoint liefert standardmäßig 100 Items pro Seite) und der JSON-Response enthält einen `__next` Link auf die nächste Seite, falls weitere Einträge vorhanden sind \[[stackoverflow.com](https:\stackoverflow.com\questions\26063068\sharepoint-2013-rest-api-not-returning-all-items-for-a-list#:~:text=If%20I%20try%20to%20gather,get%20the%20next%20100%20items)\]. Die While-Schleife iteriert, bis kein `__next` mehr vorhanden ist. Schließlich werden alle gesammelten Items in einen Pandas DataFrame `df` überführt.

> **Warum WS-Trust heute problematisch ist:** Dieser Workflow umgeht OAuth komplett und erfordert statische Benutzeranmeldeinformationen. In modernen Tenants mit **Modern Authentication** ist direkte WS-Trust Auth oft deaktiviert. Zudem funktioniert dies nicht mit Multifaktor-Authentifizierung. Microsoft empfiehlt dringend, auf OAuth 2.0-basierte Authentifizierung umzusteigen, da Basic/Legacy-Auth **keinen ausreichenden Schutz der Anmeldedaten** bietet[learn.microsoft.com](https:\learn.microsoft.com\en-us\sharepoint\technical-reference\basic-auth-is-being-deprecated#:~:text=Basic%20authentication%20doesn%27t%20provide%20confidentiality,as%20soon%20as%20possible).

### 1.1.2 | Forms-Based Authentication (FBA)

Forms-Based Authentication (FBA) ist eine **formularbasierte Anmeldung**, typischerweise in SharePoint On-Premises genutzt, wenn ein _nicht Windows_\-Identitätsanbieter (z.B. eine Membership-Provider-Datenbank) eingebunden ist. Der Benutzer gibt Nutzername/Passwort in ein Webformular ein, das System authentifiziert und setzt einen Auth-Cookie. Aus Clientsicht kann man dies simulieren, indem man den **Login-POST** an die entsprechende Seite automatisiert durchführt.

**Einschränkung:** SharePoint **Online** nutzt keine klassische FBA (es verwendet Claims-basierte Auth mit Azure AD). Dieses Beispiel würde dort nicht funktionieren. FBA gilt nur für selbst gehostete Umgebungen, wo ein solcher Anmelde-Endpunkt konfiguriert ist. Zudem ist FBA ohne HTTPS unsicher, da das Passwort im Klartext übersendet würde – dieses Szenario sollte man heute möglichst vermeiden oder nur via vertrauenswürdige Provider einsetzen.

**Codebeispiel:** (schematisch, da die konkreten Feldnamen je nach Loginseite variieren können)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Konfiguration laden
config_path = r"C:\python\Scripts\config.json"
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
USERNAME = config["sharepoint"]["username"]
PASSWORD = config["sharepoint"]["password"]

SITE_URL  = "https://<IhreDomain>.sharepoint.com/sites/<IhrSeitenname>"  # Bei FBA eher On-Prem URL
LIST_NAME = "<Name Ihrer Liste>"

session = requests.Session()
# 1. Login-Seite aufrufen, um Formularfelder und Cookies zu erhalten
login_page_url = f"{SITE_URL}/_login/default.aspx?ReturnUrl=%2F"  # FBA Login Seite (Standardpfad)
resp = session.get(login_page_url)
if resp.status_code != 200:
    raise Exception("Login-Seite konnte nicht geladen werden.")

# Hidden Fields (z.B. __VIEWSTATE, __EVENTVALIDATION) aus dem HTML parsen
soup = BeautifulSoup(resp.text, "html.parser")
viewstate = soup.find("input", {"name": "__VIEWSTATE"})["value"] if soup.find("input", {"name": "__VIEWSTATE"}) else None
eventval = soup.find("input", {"name": "__EVENTVALIDATION"})["value"] if soup.find("input", {"name": "__EVENTVALIDATION"}) else None

# 2. Login-Formulardaten zusammenstellen (Benutzereingaben + versteckte Felder)
form_data = {
    "ctl00$PlaceHolderMain$signInControl$txtUsername": USERNAME,
    "ctl00$PlaceHolderMain$signInControl$txtPassword": PASSWORD,
    "ctl00$PlaceHolderMain$signInControl$btnSignIn": "Sign In"
}
if viewstate: form_data["__VIEWSTATE"] = viewstate
if eventval: form_data["__EVENTVALIDATION"] = eventval

# 3. POST an die Login-Seite (mit Session, damit Cookies erhalten bleiben)
post_resp = session.post(login_page_url, data=form_data)
if post_resp.status_code not in (200, 302):
    raise Exception("FBA Login fehlgeschlagen.")

# Prüfen, ob Login erfolgreich (z.B. durch Vorhandensein eines Auth-Cookies)
if not any(c.name.startswith("FedAuth") for c in session.cookies):
    raise Exception("Anmeldung nicht erfolgreich - Auth-Cookie fehlt.")

# 4. Authentifizierte Anfrage an die SharePoint REST API
api_url = f"{SITE_URL}/_api/web/Lists/GetByTitle('{LIST_NAME}')/Items"
all_items = []
while True:
    resp = session.get(api_url, headers={"Accept": "application/json"})
    data = resp.json()
    items = data.get('d', {}).get('results', [])
    all_items.extend(items)
    next_url = data.get('d', {}).get('__next')
    if next_url:
        api_url = next_url
    else:
        break

df = pd.DataFrame(all_items)
print(f"{len(df)} Einträge geladen via FBA")

**Erläuterung:** Wir nutzen eine `requests.Session`, um Cookies zu behalten. Zunächst laden wir die Anmeldeseite (`_login/default.aspx` für FBA). Mit BeautifulSoup extrahieren wir versteckte Felder wie `__VIEWSTATE` und `__EVENTVALIDATION`, die SharePoint für Postbacks benötigt. Dann füllen wir das Formular mit Username, Password und dem `Sign In`\-Button-Feld und senden einen POST. Bei Erfolg sollte SharePoint einen Authentifizierungscookie (z.B. **FedAuth**) setzen. Anschließend rufen wir wieder den REST-API-Endpunkt der Liste ab, diesmal mit `session.get` (damit der Cookie mitgeschickt wird). Die Paging-Logik entspricht der aus A1.1.

**Warum dieses Beispiel möglicherweise nicht funktioniert:** In SharePoint Online gibt es diese Form-basierte Anmeldeseite so nicht – die Authentifizierung läuft über Azure AD oder MSOL. Dieses Skript würde in einer reinen Cloud-Umgebung also fehlschlagen. Für On-Premises mit FBA hängt der Erfolg davon ab, dass man die **korrekten Formularfeldnamen** verwendet – hier haben wir vermutet, dass die Standard-Login WebPart Felder `txtUsername`, `txtPassword` etc. heißen (was in SharePoint 2013-2016 Standard war). In einer angepassten Login-Seite müssten die Feldnamen entsprechend angepasst werden. Generell ist das **Screen Scraping** eines Login-Formulars anfällig für Änderungen und sollte möglichst vermieden werden. Stattdessen sollte man auch hier moderne Authentifizierung (z.B. ADFS mit OAuth oder die Verwendung der Graph API) bevorzugen.

### 1.1.3 | Basic Authentication (Benutzername/Passwort im Header)

Die einfachste, aber unsicherste Methode ist **Basic Auth**. Hierbei werden bei jedem HTTP-Request die Anmeldedaten (User und Passwort) Base64-kodiert im `Authorization` Header mitgesendet. SharePoint Server konnte (sofern konfiguriert) Basic Auth annehmen, jedoch **überträgt dies das Passwort im Klartext** (nur durch Base64 verschleiert) und sollte nur über SSL erfolgen[learn.microsoft.com](https://learn.microsoft.com/en-us/sharepoint/technical-reference/basic-auth-is-being-deprecated#:~:text=Basic%20authentication%20doesn%27t%20provide%20confidentiality,as%20soon%20as%20possible).

**In SharePoint Online** wird Basic Auth **nicht unterstützt**. Microsoft hat Basic Auth für Office 365 Dienste deaktiviert bzw. Security Defaults blockieren solche Anfragen standardmäßig[blog.admindroid.com](https://blog.admindroid.com/basic-authentication-deprecation-in-exchange-online/#:~:text=Deprecation%3F%20blog,due%20to%20security%20defaults). D.h. ein Basic Auth Versuch gegen `https://tenant.sharepoint.com` wird mit einem 401 scheitern. Für On-Premises SharePoint muss Basic Auth serverseitig explizit erlaubt sein (und wird dort ebenfalls zunehmend als veraltet markiert[learn.microsoft.com](https://learn.microsoft.com/en-us/sharepoint/technical-reference/basic-auth-is-being-deprecated#:~:text=Summary%3A%20Basic%20authentication%20is%20currently,from%20SharePoint%20Server%20Subscription%20Edition)).

**Codebeispiel:** (Dieser Aufruf funktioniert nur, wenn der Server Basic Auth erlaubt – bei SharePoint Online erhält man einen Fehler.)

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd

# Konfiguration laden
config_path = r"C:\python\Scripts\config.json"
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
USERNAME = config["sharepoint"]["username"]
PASSWORD = config["sharepoint"]["password"]

SITE_URL  = "https://<IhreDomain>.sharepoint.com/sites/<IhrSeitenname>"
LIST_NAME = "<Name Ihrer Liste>"

# Einfacher GET mit Basic Auth Header
api_url = f"{SITE_URL}/_api/web/Lists/GetByTitle('{LIST_NAME}')/Items"
all_items = []
while True:
    resp = requests.get(api_url, auth=HTTPBasicAuth(USERNAME, PASSWORD),
                        headers={"Accept": "application/json"})
    if resp.status_code == 401:
        raise Exception("Basic Auth fehlgeschlagen oder nicht unterstützt (HTTP 401)")
    data = resp.json()
    items = data.get('d', {}).get('results', [])
    all_items.extend(items)
    next_url = data.get('d', {}).get('__next')
    if next_url:
        api_url = next_url
    else:
        break

df = pd.DataFrame(all_items)
print(f"{len(df)} Einträge geladen via Basic Auth")


**Erläuterung:** Hier nutzen wir die `requests`\-Bibliothek mit `HTTPBasicAuth(USERNAME, PASSWORD)`, was automatisch den Header `Authorization: Basic <Base64>` hinzufügt. Ansonsten ist die Logik wieder gleich: Abruf der Items, prüfen auf `'__next'` und iterieren.

**Warum Basic Auth in Office 365 nicht funktioniert:** Office 365 (SharePoint Online) akzeptiert keine Basic Auth Anfragen mehr, weil sie auf **Modern Authentication (OAuth 2.0)** umgestellt haben. Basic Auth ist ein _Legacy Protocol_, das in Cloud-Diensten aus Sicherheitsgründen abgeschaltet wurde[blog.admindroid.com](https://blog.admindroid.com/basic-authentication-deprecation-in-exchange-online/#:~:text=Deprecation%3F%20blog,due%20to%20security%20defaults). In der Regel erhält man eine Fehlermeldung oder Umleitungs-HTML zu einer Login-Seite, anstatt von Daten. Zudem sind Basic-Auth-Anfragen anfällig für Man-in-the-Middle-Angriffe, da die Zugangsdaten bei jeder Anfrage mitgesendet werden[learn.microsoft.com](https://learn.microsoft.com/en-us/sharepoint/technical-reference/basic-auth-is-being-deprecated#:~:text=Basic%20authentication%20doesn%27t%20provide%20confidentiality,as%20soon%20as%20possible). In On-Prem Umgebungen könnte obiger Code funktionieren, sofern Basic Auth in den **Authentifizierungsanbietern** aktiviert ist – dies ist jedoch wie gezeigt deprecated und sollte durch Claims-basierte Authentifizierung ersetzt werden.

## 1.2. Moderne Authentifizierung (SharePoint REST)

Im modernen Ansatz erfolgt die Authentifizierung über **OAuth 2.0** mit Azure Active Directory. Anstatt Benutzerpasswörter direkt an SharePoint zu senden, erhält der Client einen **Access Token** von Azure AD, den er im HTTP-Header mitgibt (`Authorization: Bearer <Token>`[learn.microsoft.com](https://learn.microsoft.com/en-us/graph/api/site-getbypath?view=graph-rest-1.0#:~:text=Name%20Description%20Authorization%20Bearer%20,more%20about%20authentication%20and%20authorization)). SharePoint überprüft das Token (Issuer, Gültigkeit, Berechtigungen) und liefert dann die geschützten Daten. Es gibt zwei grundlegende Szenarien:

- **Delegierte Berechtigungen (A2.1)**: Ein Benutzer autorisiert die Anwendung, **in seinem Kontext** auf die API zuzugreifen. Beispiel: Device Code Flow, wo der Nutzer sich interaktiv anmeldet. Das Access Token repräsentiert den Benutzer und seine Berechtigungen.
    
- **App-Only (Anwendungsszenario, A2.2)**: Eine _Server-zu-Server_ Authentifizierung ohne Benutzer, bei der die App selbst Berechtigungen hat (entweder via Client Secret oder Zertifikat). Das Token repräsentiert eine Anwendung (Serviceprincipal) mit vordefinierten Rechten.
    

Für SharePoint gibt es hier zwei Möglichkeiten:

1. **Azure AD mit Graph-Berechtigungen** – Die App erhält Graph-Permissions wie `Sites.Read.All`. Damit kann man die SharePoint-Daten _über die Graph API_ abrufen (siehe Abschnitt B).
    
2. **Azure AD mit SharePoint-spezifischen Berechtigungen** – Man kann einer App auch direkt SharePoint API-Rechte geben (in der API-Berechtigungsliste als "Office 365 SharePoint Online" angeführt). Dies ermöglicht, ein Token mit Audience `sharepoint.com` zu erhalten und die **SharePoint REST API direkt** aufzurufen. In unseren Beispielen verwenden wir diesen direkten Weg. (Alternativ könnte man immer Graph verwenden – was oft empfehlenswerter ist –, aber wir zeigen es hier getrennt.)
    

> **Hinweis:** Microsoft bevorzugt klar die Nutzung der Graph API für neue Entwicklungen. Die direkte Verwendung von SharePoint REST mit Azure AD Token funktioniert zwar (nach entsprechender Berechtigungserteilung), ist aber weniger gut dokumentiert. In vielen Fällen ist es sinnvoll, statt `/_api` Endpunkten die Graph Endpunkte zu nutzen. Wir demonstrieren es dennoch, um die Äquivalenz zu zeigen.

### 1.2.1 | Delegierte Berechtigungen – Device Code Flow (Benutzeranmeldung)

Der **Device Code Flow** ist eine OAuth-Flussvariante, die für CLI-Tools oder Scripts ohne Browser-UI geeignet ist. Dabei wird ein Code angezeigt, den der Benutzer auf einem zweiten Gerät oder im Browser bestätigen muss (bei `https://microsoft.com/devicelogin`). Nach erfolgreicher Authentifizierung erhält das Script ein Access Token im Namen des Benutzers.

**Voraussetzung:** In Azure AD muss die App-Registrierung eine **delegierte Berechtigung** für SharePoint erhalten haben (z.B. _Alle Websites lesen_). In der Praxis gibt es für SharePoint Online z.B. die delegierte Berechtigung _"Have full control of all site collections"_ oder ähnliche, die man in der App hinzufügen und (als Admin) bestätigen muss. Alternativ genügt es auch, Graph-Delegated-Permissions (Sites.Read.All) zu vergeben und dann Graph zu nutzen (siehe B2.1). Hier nehmen wir an, die App hat entsprechende Berechtigung für direktes SharePoint.

**Codebeispiel:** Device Code Flow mit MSAL (Microsoft Authentication Library for Python):

In [ ]:
import msal
import requests
import pandas as pd

# Config laden
config_path = r"C:\python\Scripts\config.json"
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
TENANT_ID = config["azuread"]["tenant_id"]
CLIENT_ID = config["azuread"]["client_id"]
# CLIENT_SECRET wird hier nicht gebraucht, da Public Client (User Login)

SITE_URL  = "https://<IhreDomain>.sharepoint.com/sites/<IhrSeitenname>"
LIST_NAME = "<Name Ihrer Liste>"

# Public Client Application erstellen (ohne Secret)
authority_url = f"https://login.microsoftonline.com/{TENANT_ID}"
app = msal.PublicClientApplication(CLIENT_ID, authority=authority_url)

# Device Code Flow starten
scopes = [f"{SITE_URL}/.default"]  # alle Delegated-Scopes für SharePoint-API, die App hat
flow = app.initiate_device_flow(scopes=scopes)
print(flow["message"])  # Anweisungen für den Benutzer ausgeben (Code eingeben auf Website)
result = app.acquire_token_by_device_flow(flow)
if "access_token" not in result:
    raise Exception("Konnte kein Token erhalten: " + str(result.get("error")))

token = result["access_token"]
headers = {"Authorization": "Bearer " + token, "Accept": "application/json"}

# List Items abrufen mit dem Access Token
api_url = f"{SITE_URL}/_api/web/Lists/GetByTitle('{LIST_NAME}')/Items"
all_items = []
while True:
    resp = requests.get(api_url, headers=headers)
    data = resp.json()
    items = data.get('d', {}).get('results', [])
    all_items.extend(items)
    next_url = data.get('d', {}).get('__next')
    if next_url:
        api_url = next_url
    else:
        break

df = pd.DataFrame(all_items)
print(f"{len(df)} Einträge geladen (delegiert, als Benutzer)")

**Erläuterung:** Wir nutzen `msal.PublicClientApplication` für den Device Code Flow. Die `scopes` setzen wir hier auf `"{SITE_URL}/.default"`. Dieser Scope-Wert bedeutet "alle delegierten Berechtigungen für die App, die zur Resource SharePoint (Site URL) gehören". **Achtung:** In der Praxis ist es unter Umständen nötig, statt `SITE_URL` eine generischere Resource zu verwenden, da Azure AD nicht für jede einzelne Site scopes definiert. Möglicherweise wäre `"https://<IhreDomain>.sharepoint.com/AllSites.Read"` ein passender Scope-Name (falls angeboten), oder man verwendet gleich Graph-Permissions. Der `.default`\-Scope-Trick funktioniert primär für **Application**\-Permissions; für delegierte könnte man auch einen konkreten OIDC-Scope angeben, wenn vorhanden. Hier nehmen wir vereinfachend an, es klappt so – in Wirklichkeit würde man eher Graph nehmen oder den Auth-Code-Flow mit Redirect verwenden.

Nach Initiierung des Device Flows wird eine Meldung mit einem Code ausgegeben (`flow["message"]`), z.B.: _"Öffnen Sie https://microsoft.com/devicelogin und geben Sie den Code ABC-DEF-GHI ein."_ Der Benutzer muss das tun und sich einloggen. MSAL wartet währenddessen in `acquire_token_by_device_flow` auf Abschluss. Danach erhalten wir `access_token`. Dieses verwenden wir, um den `Authorization: Bearer <Token>` Header zu setzen. Der nachfolgende GET-Aufruf an `/_api/web/Lists/...` entspricht dem aus vorherigen Beispielen, nur dass keine Cookies oder BasicAuth mehr nötig sind – das OAuth-Token genügt.

**Auswirkung:** SharePoint prüft das Token. Wenn der Benutzer die erforderlichen Rechte auf der Site/Liste hat und die App die entsprechende Delegated Permission besitzt, wird der Call erfolgreich sein und die Daten zurückliefern. Die Paging-Mechanik bleibt unverändert. Der DataFrame wird am Ende mit allen Listeneinträgen gefüllt.

**Häufige Probleme:** Falls das Token nicht akzeptiert wird, kann es daran liegen, dass:

- Die **Scopes/Berechtigungen** nicht korrekt sind. (E.g. falscher Resource-URI oder App hat keine SharePoint-Delegated-Permission.)
    
- Das Token eine falsche Audience hat. (Ein Graph-Token würde z.B. mit `aud": "https://graph.microsoft.com"` im Token nicht von `sharepoint.com` akzeptiert – Tokens sind normalerweise nur für die jeweilige Resource gültig.)
    
- In neueren Setups wird delegierter Zugriff auf SharePoint-Daten oft _über Graph_ realisiert. Microsoft hat z.B. angekündigt, dass direkte Azure AD Tokens mit `user_impersonation` Scope für SPO evtl. nicht mehr unterstützt werden[stackoverflow.com](https://stackoverflow.com/questions/76629048/accessing-sharepoint-via-visual-studio-app-is-not-allowed-to-call-spo-with-user#:~:text=The%20error%20,access%20SharePoint%20using%20invalid%20permissions). Die Empfehlung ist klar: Für benutzerbezogene Zugriffe auf SharePoint Online -\> verwende Microsoft Graph[stackoverflow.com](https://stackoverflow.com/questions/76629048/accessing-sharepoint-via-visual-studio-app-is-not-allowed-to-call-spo-with-user#:~:text=,error).
    

Im Zweifel sollte man daher stattdessen zum **Graph-API-Gerätecode** greifen (siehe unten B2.1), da dieser Weg offiziell unterstützt ist.

### 1.2.2 | App-Only (Client Credential Flow) – mit Client Secret oder Zertifikat

Im App-Only Szenario agiert unsere Python-Anwendung eigenständig, ohne Benutzer. Sie authentifiziert sich mit ihren eigenen Anmeldedaten (Client-ID + Secret oder Zertifikat) bei Azure AD und bekommt ein **App-Access-Token**. Dieses Token hat die in der App-Registrierung definierten **Anwendungsberechtigungen** (Application Permissions), die ein Administrator zuvor **admin-consented** haben muss. Für SharePoint könnte das z.B. _"Sites.Read.All (Application)"_ oder _"Sites.FullControl.All"_ sein – entweder über Graph oder über SharePoint direkt.

In klassischen SharePoint Add-Ins gab es auch ein separates Modell (App-Reg via `/_layouts/15/appregnew.aspx` und Auth über ACS, den Azure Access Control Service). Dieses ACS-Modell wird allerdings auch in Zukunft abgeschaltet. Deshalb verwenden wir hier ebenfalls Azure AD (Microsoft Entra ID) als STS.

Wir zeigen zwei Varianten, die sich nur in der Art der Client-Credentials unterscheiden:

- **A2.2.1 Mit Client Secret:** einfacher, aber das Secret ist ein String der in der Konfig liegt. Sollte sicher verwaltet werden, da es wie ein Passwort fungiert.
    
- **A2.2.2 Mit Zertifikat:** sicherer, da die App in Azure AD ein Zertifikat hinterlegt hat. Das Script nutzt den privaten Schlüssel, um sich auszuweisen. Das Zertifikat muss nicht im Code stehen (nur Pfad nötig), und es kann in Azure jederzeit invalidiert werden. Diese Methode ist geeignet für höhere Sicherheit oder wenn man keine Secrets in Konfigs ablegen möchte.

In [ ]:
# Codebeispiel 1.2.2.1 – Client Credential mit Secret:
import msal
import requests
import pandas as pd

# Config laden
config_path = r"C:\python\Scripts\config.json"
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
TENANT_ID     = config["azuread"]["tenant_id"]
CLIENT_ID     = config["azuread"]["client_id"]
CLIENT_SECRET = config["azuread"]["client_secret"]

SITE_URL  = "https://<IhreDomain>.sharepoint.com/sites/<IhrSeitenname>"
LIST_NAME = "<Name Ihrer Liste>"

# Confidential Client (mit Secret) erstellen
authority = f"https://login.microsoftonline.com/{TENANT_ID}"
app = msal.ConfidentialClientApplication(client_id=CLIENT_ID,
                                        client_credential=CLIENT_SECRET,
                                        authority=authority)

# Token via Client Credentials Flow holen (Scope: SharePoint Site)
result = app.acquire_token_for_client(scopes=[f"{SITE_URL}/.default"])
if "access_token" not in result:
    raise Exception("Token-Erhalt fehlgeschlagen: " + str(result.get("error")))

token = result["access_token"]
headers = {"Authorization": "Bearer " + token, "Accept": "application/json"}

# API-Call an SharePoint
api_url = f"{SITE_URL}/_api/web/Lists/GetByTitle('{LIST_NAME}')/Items"
all_items = []
while True:
    resp = requests.get(api_url, headers=headers)
    data = resp.json()
    items = data.get('d', {}).get('results', [])
    all_items.extend(items)
    next_url = data.get('d', {}).get('__next')
    if next_url:
        api_url = next_url
    else:
        break

df = pd.DataFrame(all_items)
print(f"{len(df)} Einträge geladen (App-Only mit Secret)")


In [ ]:
# Codebeispiel 1.2.2.2 – Client Credential mit Zertifikat:
import msal
import requests
import pandas as pd

# Config laden
config_path = r"C:\python\Scripts\config.json"
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
TENANT_ID  = config["azuread"]["tenant_id"]
CLIENT_ID  = config["azuread"]["client_id"]
CERT_PATH  = config["azuread"]["cert_path"]        # Pfad zur PEM-Datei mit privatem Schlüssel
CERT_THUMBPRINT = config["azuread"]["cert_thumbprint"]  # Zertifikat-Thumbprint wie in AAD registriert

SITE_URL  = "https://<IhreDomain>.sharepoint.com/sites/<IhrSeitenname>"
LIST_NAME = "<Name Ihrer Liste>"

# Zertifikat einlesen
with open(CERT_PATH, "r") as f:
    private_key = f.read()

app = msal.ConfidentialClientApplication(
    client_id=CLIENT_ID,
    authority=f"https://login.microsoftonline.com/{TENANT_ID}",
    client_credential={"private_key": private_key, "thumbprint": CERT_THUMBPRINT}
)

result = app.acquire_token_for_client(scopes=[f"{SITE_URL}/.default"])
if "access_token" not in result:
    raise Exception("Token-Erhalt fehlgeschlagen: " + str(result.get("error")))

token = result["access_token"]
headers = {"Authorization": "Bearer " + token, "Accept": "application/json"}

# API Call (gleich wie oben)
api_url = f"{SITE_URL}/_api/web/Lists/GetByTitle('{LIST_NAME}')/Items"
all_items = []
while True:
    resp = requests.get(api_url, headers=headers)
    data = resp.json()
    items = data.get('d', {}).get('results', [])
    all_items.extend(items)
    next_url = data.get('d', {}).get('__next')
    if next_url:
        api_url = next_url
    else:
        break

df = pd.DataFrame(all_items)
print(f"{len(df)} Einträge geladen (App-Only mit Zertifikat)")


**Erläuterung:** Beide Varianten nutzen `msal.ConfidentialClientApplication.acquire_token_for_client`. Wir geben als Scope wiederum `SITE_URL/.default` an, was bedeutet: Fordere ein Token an, das alle **Anwendungsberechtigungen** für die Resource _SharePoint-Site_ enthält, die für diese App registriert sind. In Azure AD muss dazu z.B. eine **Anwendungsberechtigung** wie _"Sites.Read.All"_ unter **SharePoint** hinzugefügt und vom Admin **grantet** worden sein. Ist dies erfüllt, liefert Azure AD ein Access Token (mit dem Issuer/Audience passend für SharePoint Online).

- In A2.2.1 authentifizieren wir uns mit dem Client-Secret (Zeile `client_credential=CLIENT_SECRET`).
    
- In A2.2.2 übergeben wir ein Dictionary mit `private_key` und `thumbprint`. Die App-Registrierung muss das öffentliche Zertifikat (entsprechend dem privaten Schlüssel) hinterlegt haben. MSAL signiert dann eine Assertion mit dem Private Key, um das Token zu erhalten[learn.microsoft.com](https://learn.microsoft.com/en-us/entra/msal/python/advanced/client-credentials#:~:text=When%20the%20application%20is%20registered,passing%20the%20scope%20as%20parameter)[learn.microsoft.com](https://learn.microsoft.com/en-us/entra/msal/python/advanced/client-credentials#:~:text=client_credential%20%3D%20%7B%20,).
    

Nach Erhalt des Tokens werden die Listendaten genauso abgefragt wie zuvor, mit dem `Authorization: Bearer` Header. Paging und DataFrame-Bau sind identisch.

**Wichtig:** Das Access Token hat nun die Rechte, die der App zugewiesen wurden, unabhängig von einem Benutzer. Beispielsweise könnte eine App mit _Sites.Read.All_ an **alle** Sites gelangen, egal welcher User es ausführt. Deshalb ist die **Zugriffskontrolle** hier anders: man sollte sicherstellen, dass solche App-Only Credentials sicher gespeichert sind und die App-Registrierung nur minimal nötige Rechte hat.

**Mögliche Fehlerursachen:**

- 403 **Zugriff verweigert**: Falls die App zwar ein Token hat, aber keine Rechte auf die konkrete Site. Bei SharePoint-Anwendungsrechten ist es manchmal erforderlich, der App mittels SharePoint Admin Center App-Profile oder Appinv.xml _Tenant Full Control_ zu geben, je nach Auth-Modell. (Bei Graph-Permissions sollte Sites.Read.All eigentlich ausreichen.)
    
- 401 **Unauthorized**: Das könnte bedeuten, das Token wird nicht akzeptiert – z.B. weil Scope/Audience falsch ist. Evtl. muss man statt `SITE_URL/.default` einen anderen Resource-Bezeichner nehmen. In einigen Szenarien nutzt man auch einfach Graph (siehe B2.2).
    
- Wenn **MFA** im Spiel ist, betrifft das App-Only nicht (da kein User).
    
- Der **Tenant** könnte Legacy ACS App-Only Tokens abgeschaltet haben. (Wir nutzen aber Azure AD, daher nicht relevant in diesem Code.)
    

Abschließend sei erwähnt: Mit diesen modernen Methoden (Delegated und App-Only) haben wir sehr ähnliche Abläufe wie im folgenden Graph-Abschnitt, nur dass wir hier direkt die SharePoint-REST-URL ansprechen. Oft ist es bequemer, gleich die Graph-API zu verwenden, die wir nun betrachten.

# 2 | Microsoft Graph REST API

Die Microsoft Graph API bietet einen einheitlichen Endpunkt (`https://graph.microsoft.com`) für viele Microsoft 365 Dienste, darunter SharePoint. Über Graph kann man z.B. auf **Sites, Lists, List Items** etc. zugreifen. Intern erledigt Graph die Authentifizierung wiederum mit Azure AD Tokens. Graph **akzeptiert ausschließlich OAuth 2.0 Bearer Tokens** – andere Auth-Methoden funktionieren hier nicht[learn.microsoft.com](https://learn.microsoft.com/en-us/graph/api/site-getbypath?view=graph-rest-1.0#:~:text=Name%20Description%20Authorization%20Bearer%20,more%20about%20authentication%20and%20authorization).

Wir zeigen analog die Fälle für Legacy (B1) und modern (B2) in Bezug auf Graph. Allerdings sind die Legacy-Methoden bei Graph größtenteils **nicht anwendbar**: Graph unterstützt _keine_ direkte Username/Password Auth ohne Token. Die Beispiele B1.x dienen also eher dazu zu demonstrieren, dass man es versuchen könnte, aber ein **Fehler** resultiert.

Für die modernen Fälle B2.x nutzen wir weitgehend das gleiche Vorgehen wie A2.x, aber rufen die **Graph-Endpoints** auf (anstatt SharePoint-Endpoints). Wichtig: In Graph unterscheiden sich die URL-Pfade etwas, und man muss i.d.R. mit **Site-IDs** oder Pfaden hantieren.

Bevor wir in Code eintauchen, zunächst ein kurzer Überblick, wie man auf eine SharePoint-Liste via Graph zugreift:

- Man braucht die **Site ID** oder kann die Site per Pfad angeben.
    
- Dann die **List ID** oder den Listennamen.
    
- Graph-URL-Beispiel:  
    `GET https://graph.microsoft.com/v1.0/sites/<hostname>:/sites/<SitePfad>:/lists/<ListName>/items?expand=fields`
    

Hier bedeuten:

- `<hostname>`: z.B. `contoso.sharepoint.com`
    
- `<SitePfad>`: z.B. `sites/meineTeamseite`
    
- `<ListName>`: Anzeigename der Liste (funktioniert, sofern eindeutig)
    

Alternativ kann man zunächst die Site-ID ermitteln mit `GET /sites/<hostname>:/sites/<SitePfad>` und dann `GET /sites/<Site-ID>/lists/<List-ID>/items`. In unseren Scripts verwenden wir aber den Pfad-Kurzschluss, weil wir `SITE_URL` und `LIST_NAME` schon kennen.

Graph liefert Listeneinträge in der Regel als Array von `listItem` Objekten, die die Daten unter `fields` enthalten. Wir werden diese extrahieren und ins DataFrame übernehmen. Pagination gibt Graph über `@odata.nextLink` an.

## 2.1 | Legacy-Authentifizierung (Graph API)

Wie erwähnt, hat Graph keine Legacy-Auth-Modi. Trotzdem besprechen wir kurz, was passiert, wenn man es versucht:

### 2.1.1 | WS-Trust / SAML Token gegen Graph

Es ist **nicht möglich**, ein via WS-Trust erworbenes SAML/FedAuth-Cookie für Graph zu nutzen. Graph erwartet ein OAuth Bearer Token im Header und kennt keine FedAuth-Cookies. Selbst wenn man mit dem WS-Trust-Ansatz aus A1.1 an SharePoint gekommen ist, kann man damit **nicht** die Graph-URL ansprechen – man würde schlicht eine HTML-Login-Aufforderung oder 401 erhalten.

Ein hypothetisches Beispiel (nicht funktional):

In [ ]:
# Annahme: Wir haben saml_token aus A1.1 und FedAuth Cookie
graph_url = "https://graph.microsoft.com/v1.0/sites/contoso.sharepoint.com:/sites/meineTeamseite:/lists/MyList"
resp = requests.get(graph_url, cookies=cookies)  # cookies von SharePoint
print(resp.status_code, resp.text)


Graph ignoriert die SharePoint-Cookies, weil es ein separater Dienst ist. Das Ergebnis wäre vermutlich ein **401 Unauthorized**, mit einer Fehlermeldung im Body wie _"Access token is missing or invalid."_ Graph **erzwingt** ein gültiges Azure AD Access Token. Daher lohnt sich kein weiteres Eingehen auf WS-Trust für Graph – man muss auf OAuth umsteigen.

### 2.1.2 | Forms-Based Auth (Login-POST) gegen Graph

Graph hat keine HTML-Formular-Loginseite, da die Authentifizierung ausschließlich über das Microsoft Identity System läuft. Es gibt also keine Entsprechung zu FBA. Jeder Versuch, Graph mit Session/Cookie basierter Auth zu nutzen, wird fehlschlagen. Selbst wenn man sich in einem Browser bei SharePoint eingeloggt hat, sind die Cookies nicht für `graph.microsoft.com` gültig.

**Fazit:** Nicht anwendbar.

### 2.1.3 | Basic Auth gegen Graph

Basic Auth wird von Graph nicht akzeptiert. Wenn man versucht, einen Graph-Request mit `auth=HTTPBasicAuth(user, pass)` abzusetzen, erhält man ebenfalls einen **401**. Graph gibt dann meist eine JSON-Fehlermeldung zurück, zum Beispiel:

In [ ]:
{
  "error": {
    "code": "InvalidAuthenticationToken",
    "message": "Access token is missing or invalid."
  }
}

Ein kurzer Demonstrationscode:

In [ ]:
resp = requests.get("https://graph.microsoft.com/v1.0/sites/root", 
                    auth=HTTPBasicAuth("user@domain.com", "passw0rd"))
print(resp.status_code)
print(resp.text)

Dies würde Status `401` und eine Meldung ähnlich obigem JSON zeigen. Graph **verlangt** den `Authorization: Bearer <token>` Header[learn.microsoft.com](https://learn.microsoft.com/en-us/graph/api/site-getbypath?view=graph-rest-1.0#:~:text=Name%20Description%20Authorization%20Bearer%20,more%20about%20authentication%20and%20authorization). Deshalb müssen wir uns an die modernen Methoden halten, um Graph zu nutzen.

## 2.2 | Moderne Authentifizierung (Graph API)

Hier kommen wir zum eigentlichen empfohlenen Weg: Mit Azure AD Token die Graph API aufzurufen. Die Patterns sind sehr ähnlich zu 1.2.1 und 1.2.2, lediglich die **Scopes** bzw. Resource ändern sich, und die URL, die abgefragt wird, ist die Graph-URL.

### 2.2.1 | Delegierte Berechtigungen – Device Code (Graph)

Im Graph-Kontext holen wir ein Token mit z.B. Scope `Sites.Read.All` (delegated) und rufen die List Items über den Graph-Endpoint ab.

In [ ]:
import msal
import requests
import pandas as pd

config_path = r"C:\python\Scripts\config.json"
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
TENANT_ID = config["azuread"]["tenant_id"]
CLIENT_ID = config["azuread"]["client_id"]

# Angenommen, App hat Delegated Permission "Sites.Read.All" für Graph
app = msal.PublicClientApplication(CLIENT_ID, authority=f"https://login.microsoftonline.com/{TENANT_ID}")
flow = app.initiate_device_flow(scopes=["Sites.Read.All"])
print(flow["message"])
result = app.acquire_token_by_device_flow(flow)
if "access_token" not in result:
    raise Exception("Token Error: " + result.get("error"))
token = result["access_token"]

# Graph-Endpoint für die SharePoint-Liste vorbereiten (Pfad mittels Site-URL und List Name)
hostname = "<IhreDomain>.sharepoint.com"
site_path = "sites/<IhrSeitenname>"
list_name = LIST_NAME  # denselben LIST_NAME wie oben verwenden

url = f"https://graph.microsoft.com/v1.0/sites/{hostname}:/{site_path}:/lists/{list_name}/items?expand=fields"
headers = {"Authorization": "Bearer " + token}
all_items = []
while True:
    resp = requests.get(url, headers=headers)
    data = resp.json()
    items = data.get("value", [])
    # Graph gibt die Listeneinträge in 'value' zurück; Felder innerhalb von item["fields"]
    all_items.extend(items)
    next_link = data.get("@odata.nextLink")
    if next_link:
        url = next_link  # nächste Seite abfragen
    else:
        break

# Wir extrahieren nur die 'fields', die die eigentlichen Listendaten enthalten:
records = [item["fields"] for item in all_items]
df = pd.DataFrame(records)
print(f"{len(df)} Einträge via Graph geladen.")


**Erläuterung:** Wir fordern ein Graph-Token mit dem Scope `"Sites.Read.All"` an (dies setzt voraus, der Benutzer hat interaktiv der App die Rechte gegeben, oder ein Admin hat voraus-gewährt). Der Device-Code-Flow läuft analog ab – der Benutzer loggt sich ein, MSAL gibt uns ein Token zurück. Dieses Token enthält Berechtigungen für Graph (hier: Lesen aller SharePoint Seiten/Sites) und ist vom Typ **Bearer**.

Die URL-Konstruktion: Wir nutzen die Möglichkeit, die Site per Pfad anzusprechen `sites/{hostname}:/{site-path}:/`. Dann hängen wir `/lists/{ListName}/items` an. Durch `?expand=fields` erhalten wir pro ListItem die Felder. (Ohne `expand` würde Graph nur die item IDs liefern, mit extra Abfrage nötig). In `items` sammeln wir die rohen ListItem-Objekte, aber am Ende extrahieren wir nur den `fields`\-Teil jedes Items, um einen sauberen DataFrame der Listenspalten zu erstellen.

Die Pagination erkennt man daran, dass Graph ggf. ein `"@odata.nextLink"` liefert[learn.microsoft.com](https://learn.microsoft.com/en-us/graph/api/listitem-list?view=graph-rest-1.0#:~:text=Depending%20on%20the%20number%20of,Graph%20data%20in%20your%20app). Solange dieser vorhanden ist, laden wir die nächste URL. (Graph paget standardmäßig z.B. in 100er Schritten, je nach API.)

Dieses Vorgehen ist schon die **empfohlene** Methode für Benutzerzugriffe. Sie hat gegenüber A2.1 den Vorteil, dass Microsoft Graph die API ist, die kontinuierlich erweitert wird und auf die zukünftige Entwicklung setzt. Azure AD-Consent für Graph-Permissions ist meistens einfacher zu handhaben als direkte SharePoint-Rechte.

### 2.2.2 | App-Only – Client Credentials (Graph)

Zum Abschluss das App-Only-Beispiel mit Client Secret (analog A2.2.1, aber für Graph):

In [ ]:
import msal
import requests
import pandas as pd

config_path = r"C:\python\Scripts\config.json"
with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
TENANT_ID     = config["azuread"]["tenant_id"]
CLIENT_ID     = config["azuread"]["client_id"]
CLIENT_SECRET = config["azuread"]["client_secret"]

# Confidential Client for Graph
app = msal.ConfidentialClientApplication(CLIENT_ID, authority=f"https://login.microsoftonline.com/{TENANT_ID}",
                                        client_credential=CLIENT_SECRET)
# Access Token mit Application Permission "Sites.Read.All"
result = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])
if "access_token" not in result:
    raise Exception("Token Error: " + str(result.get("error")))
token = result["access_token"]

# Graph-Request an die SharePoint-Liste
hostname = "<IhreDomain>.sharepoint.com"
site_path = "sites/<IhrSeitenname>"
list_name = LIST_NAME

url = f"https://graph.microsoft.com/v1.0/sites/{hostname}:/{site_path}:/lists/{list_name}/items?expand=fields"
headers = {"Authorization": "Bearer " + token}
all_items = []
while True:
    resp = requests.get(url, headers=headers)
    data = resp.json()
    items = data.get("value", [])
    all_items.extend(items)
    if "@odata.nextLink" in data:
        url = data["@odata.nextLink"]
    else:
        break

records = [item["fields"] for item in all_items]
df = pd.DataFrame(records)
print(f"{len(df)} Einträge via Graph App-Only geladen.")


**Erläuterung:** Wir fordern mit `scopes=["https://graph.microsoft.com/.default"]` ein Token an, das **alle** Anwendungsberechtigungen enthält, die unserer App für Graph gewährt wurden. In unserem Beispiel sollte die App mind. _Sites.Read.All (Application)_ haben. Das zurückgegebene Token erlaubt dann das Lesen aller SharePoint-Seiten über Graph ohne Benutzerkontext.

Die Abfrage-Logik ist identisch zu B2.1, lediglich dass wir hier kein Benutzerpopup haben und das Token direkt bekommen. Dieses Beispiel nutzt ein Client Secret. Für ein Zertifikat würde man `client_credential={"private_key":..., "thumbprint":...}` ähnlich wie in A2.2.2 einsetzen – das funktioniert mit Graph genauso.

**Fazit:** Die Graph-App-Only Methode ist sehr leistungsfähig für Backend-Dienste, muss aber mit Vorsicht verwendet werden: Das Token umgeht alle Benutzerprüfungen und hat evtl. Zugriff auf sehr viele Daten. Entsprechend sollte die App-Registrierung restriktiv konfiguriert und das Secret sicher verwahrt sein.

# 3 | Zusammenfassung und Best Practices

Wir haben eine Reihe von Authentifizierungsvarianten durchgespielt:

- **Legacy-Methoden (WS-Trust, FBA, Basic)**: Historisch interessant, aber in modernen Cloud-Szenarien entweder unbrauchbar oder **unsicher**. Basic Auth und SOAP-basierte Ansätze werden von Microsoft abgelehnt und teils bereits deaktiviert[learn.microsoft.com](https://learn.microsoft.com/en-us/sharepoint/technical-reference/basic-auth-is-being-deprecated#:~:text=Basic%20authentication%20doesn%27t%20provide%20confidentiality,as%20soon%20as%20possible). Für On-Premises sind FBA und NTLM noch relevant, aber auch dort sollte man mittelfristig auf Claims/OAuth umstellen.
    
- **Moderne Methoden (OAuth via Azure AD)**: Diese sind heute Standard. Für **Benutzergesteuerte** Abläufe bietet sich der Device Code Flow oder interaktive OAuth an. Für **hintergrund**\-Dienste nutzt man Client Credentials (mit Secret oder besser Zertifikat). In allen Fällen erhält man ein **Bearer Token**, das in den API-Anfragen verwendet wird[learn.microsoft.com](https://learn.microsoft.com/en-us/graph/api/site-getbypath?view=graph-rest-1.0#:~:text=Name%20Description%20Authorization%20Bearer%20,more%20about%20authentication%20and%20authorization).
    
- **SharePoint REST vs. Graph API**: Man kann SharePoint-Daten entweder über die SharePoint REST Endpunkte (`_api/web/...`) oder über Graph (`graph.microsoft.com/v1.0/sites/...`) abrufen. Graph bündelt viele Dienste und ist zukunftssicher, aber die SharePoint-spezifischen APIs bieten manchmal Funktionen, die in Graph (noch) nicht verfügbar sind[learn.microsoft.com](https://learn.microsoft.com/en-us/answers/questions/78034/how-to-connect-python-program-to-sharepoint-rest-a#:~:text=sharepoint). Je nach Anwendungsfall kann man entscheiden – oftmals wird Microsoft Graph bevorzugt, insbesondere um nicht separate Authentifizierungen pro Service handhaben zu müssen.
    

**Paging und DataFrame:** In allen Beispielen haben wir darauf geachtet, alle Seiten von Ergebnissen abzurufen. Sowohl SharePoint REST als auch Graph liefern einen Marker (`__next` bzw. `@odata.nextLink`), wenn noch nicht alle Items übertragen wurden. Die gezeigten Patterns mit einer While-Schleife lassen sich auch für andere API-Abfragen wiederverwenden[spguides.com](https://www.spguides.com/get-all-sharepoint-list-items-using-rest-api-pagination-in-power-automate/#:~:text=Get%20All%20SharePoint%20List%20Items,there%27s%20another%20page%20of%20data)[learn.microsoft.com](https://learn.microsoft.com/en-us/graph/api/listitem-list?view=graph-rest-1.0#:~:text=Depending%20on%20the%20number%20of,Graph%20data%20in%20your%20app). Schließlich werden die Ergebnisse in einen Pandas DataFrame überführt, was die Weiterverarbeitung in Python erleichtert (z.B. Analysen, Export etc.).

**Sicherheitshinweis:** Bitte legen Sie **niemals** Hardcode-Passwörter oder Secrets im Code ab. Nutzen Sie wie hier gezeigt externe Konfiguration, und schützen Sie diese angemessen (Dateisystem-Rechte, ggf. Verschlüsselung). Für echte Anwendungen sollten Secrets und Zertifikate in **Azure Key Vault** oder ähnlichen Secret-Stores liegen. Zudem ist es ratsam, regelmäßig abgelaufene Tokens nicht unkontrolliert lange gültig zu lassen (Default Gültigkeit bei Azure AD ist ca. 1 Stunde pro Access Token).

Mit den modernen Methoden und dem gezeigten Code-Gerüst sollten Sie in der Lage sein, auf SharePoint-Listen zuzugreifen. Bei der Implementierung in der Praxis ist immer der erste Schritt, die nötigen **App-Registrierungen und Berechtigungen** in Azure AD korrekt einzurichten. Danach kann man mit MSAL den Auth-Fluss implementieren und die gewünschten REST-Aufrufe durchführen. So vermeiden Sie unsichere Ansätze und sind auf dem von Microsoft vorgesehenen Weg für SharePoint-Integrationen.

# Quellen

- <span style="color: var(--vscode-foreground);">Microsoft Doku – SharePoint Online Authentifizierung (aktiv vs. passiv)</span><span class="" data-state="closed" style="color: var(--vscode-foreground);"><span class="ms-1 inline-flex max-w-full items-center relative top-[-0.094rem] animate-[show_150ms_ease-in]" data-testid="webpage-citation-pill"><a href="https://sharepoint.stackexchange.com/questions/139209/sharepoint-rest-api-authentication-with-saml#:~:text=SharePoint%20Online%20uses%20a%20token,security%20token%20from%20the%20cookies" target="_blank" rel="noopener" alt="https://sharepoint.stackexchange.com/questions/139209/sharepoint-rest-api-authentication-with-saml#:~:text=SharePoint%20Online%20uses%20a%20token,security%20token%20from%20the%20cookies" class="flex h-4.5 overflow-hidden rounded-xl px-2 text-[9px] font-medium text-token-text-secondary! bg-[#F4F4F4]! dark:bg-[#303030]! transition-colors duration-150 ease-in-out">sharepoint.stackexchange.com</a>&nbsp;</span></span> 
- Microsoft Doku – Graph API erfordert OAuth 2.0 (Bearer Token im Header)<span class="" data-state="closed" style="color: var(--vscode-foreground);"><span class="ms-1 inline-flex max-w-full items-center relative top-[-0.094rem] animate-[show_150ms_ease-in]" data-testid="webpage-citation-pill"><a href="https://learn.microsoft.com/en-us/graph/api/site-getbypath?view=graph-rest-1.0#:~:text=Name%20Description%20Authorization%20Bearer%20,more%20about%20authentication%20and%20authorization" target="_blank" rel="noopener" alt="https://learn.microsoft.com/en-us/graph/api/site-getbypath?view=graph-rest-1.0#:~:text=Name%20Description%20Authorization%20Bearer%20,more%20about%20authentication%20and%20authorization" class="flex h-4.5 overflow-hidden rounded-xl px-2 text-[9px] font-medium text-token-text-secondary! bg-[#F4F4F4]! dark:bg-[#303030]! transition-colors duration-150 ease-in-out">learn.microsoft.com</a></span></span>
- Blogpost Joseph Velliah – **SAML Auth Flow für SharePoint Online** <span style="color: var(--vscode-foreground);"> (WS-Trust zu FedAuth Cookie)</span><span class="" data-state="closed" style="color: var(--vscode-foreground);"><span class="ms-1 inline-flex max-w-full items-center relative top-[-0.094rem] animate-[show_150ms_ease-in]" data-testid="webpage-citation-pill"><a href="https://blog.josephvelliah.com/access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=Now%20hit%20Send%20button%20to,should%20be%20something%20like%20this" target="_blank" rel="noopener" alt="https://blog.josephvelliah.com/access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=Now%20hit%20Send%20button%20to,should%20be%20something%20like%20this" class="flex h-4.5 overflow-hidden rounded-xl px-2 text-[9px] font-medium text-token-text-secondary! bg-[#F4F4F4]! dark:bg-[#303030]! transition-colors duration-150 ease-in-out">blog.josephvelliah.com</a></span></span><span class="" data-state="closed" style="color: var(--vscode-foreground);"><span class="ms-1 inline-flex max-w-full items-center relative top-[-0.094rem] animate-[show_150ms_ease-in]" data-testid="webpage-citation-pill"><a href="https://blog.josephvelliah.com/access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=The%20response%20for%20this%20request,down%20the%20values%20of%20the%C2%A0rtFa%C2%A0and%C2%A0FedAuth%C2%A0Cookies" target="_blank" rel="noopener" alt="https://blog.josephvelliah.com/access-sharepoint-online-rest-api-via-postman-with-user-context#:~:text=The%20response%20for%20this%20request,down%20the%20values%20of%20the%C2%A0rtFa%C2%A0and%C2%A0FedAuth%C2%A0Cookies" class="flex h-4.5 overflow-hidden rounded-xl px-2 text-[9px] font-medium text-token-text-secondary! bg-[#F4F4F4]! dark:bg-[#303030]! transition-colors duration-150 ease-in-out">blog.josephvelliah.com</a></span></span>
- Microsoft Q&A – **Office365-REST-Python-Client** <span style="color: var(--vscode-foreground);"> (Beispiele für App-Only Auth)</span><span class="" data-state="closed" style="color: var(--vscode-foreground);"><span class="ms-1 inline-flex max-w-full items-center relative top-[-0.094rem] animate-[show_150ms_ease-in]" data-testid="webpage-citation-pill"><a href="https://learn.microsoft.com/en-us/answers/questions/78034/how-to-connect-python-program-to-sharepoint-rest-a#:~:text=app_settings%20%3D%20,zzzzzfadfd%27%2C%20%27client_secret%27%3A%20%27Tteadsfdafdfasdff444gadfd%3D%27%2C" target="_blank" rel="noopener" alt="https://learn.microsoft.com/en-us/answers/questions/78034/how-to-connect-python-program-to-sharepoint-rest-a#:~:text=app_settings%20%3D%20,zzzzzfadfd%27%2C%20%27client_secret%27%3A%20%27Tteadsfdafdfasdff444gadfd%3D%27%2C" class="flex h-4.5 overflow-hidden rounded-xl px-2 text-[9px] font-medium text-token-text-secondary! bg-[#F4F4F4]! dark:bg-[#303030]! transition-colors duration-150 ease-in-out">learn.microsoft.com</a></span></span><span class="" data-state="closed" style="color: var(--vscode-foreground);"><span class="ms-1 inline-flex max-w-full items-center relative top-[-0.094rem] animate-[show_150ms_ease-in]" data-testid="webpage-citation-pill"><a href="https://learn.microsoft.com/en-us/answers/questions/78034/how-to-connect-python-program-to-sharepoint-rest-a#:~:text=You%20can%20use%20Office365,Only%20Credential%20authentication%20%28AuthenticationContext.ctx_auth.acquire_token_for_app%28client_id%2C%20client_secret" target="_blank" rel="noopener" alt="https://learn.microsoft.com/en-us/answers/questions/78034/how-to-connect-python-program-to-sharepoint-rest-a#:~:text=You%20can%20use%20Office365,Only%20Credential%20authentication%20%28AuthenticationContext.ctx_auth.acquire_token_for_app%28client_id%2C%20client_secret" class="flex h-4.5 overflow-hidden rounded-xl px-2 text-[9px] font-medium text-token-text-secondary! bg-[#F4F4F4]! dark:bg-[#303030]! transition-colors duration-150 ease-in-out">learn.microsoft.com</a></span></span>
- Microsoft Learn – Deprecation von Basic Auth in SharePoint<span class="" data-state="closed" style="color: var(--vscode-foreground);"><span class="ms-1 inline-flex max-w-full items-center relative top-[-0.094rem] animate-[show_150ms_ease-in]" data-testid="webpage-citation-pill"><a href="https://learn.microsoft.com/en-us/sharepoint/technical-reference/basic-auth-is-being-deprecated#:~:text=Summary%3A%20Basic%20authentication%20is%20currently,from%20SharePoint%20Server%20Subscription%20Edition" target="_blank" rel="noopener" alt="https://learn.microsoft.com/en-us/sharepoint/technical-reference/basic-auth-is-being-deprecated#:~:text=Summary%3A%20Basic%20authentication%20is%20currently,from%20SharePoint%20Server%20Subscription%20Edition" class="flex h-4.5 overflow-hidden rounded-xl px-2 text-[9px] font-medium text-token-text-secondary! bg-[#F4F4F4]! dark:bg-[#303030]! transition-colors duration-150 ease-in-out">learn.microsoft.com</a></span></span><span class="" data-state="closed" style="color: var(--vscode-foreground);"><span class="ms-1 inline-flex max-w-full items-center relative top-[-0.094rem] animate-[show_150ms_ease-in]" data-testid="webpage-citation-pill"><a href="https://learn.microsoft.com/en-us/sharepoint/technical-reference/basic-auth-is-being-deprecated#:~:text=Basic%20authentication%20doesn%27t%20provide%20confidentiality,as%20soon%20as%20possible" target="_blank" rel="noopener" alt="https://learn.microsoft.com/en-us/sharepoint/technical-reference/basic-auth-is-being-deprecated#:~:text=Basic%20authentication%20doesn%27t%20provide%20confidentiality,as%20soon%20as%20possible" class="flex h-4.5 overflow-hidden rounded-xl px-2 text-[9px] font-medium text-token-text-secondary! bg-[#F4F4F4]! dark:bg-[#303030]! transition-colors duration-150 ease-in-out">learn.microsoft.com</a></span></span>
- Stack Overflow – Fehler "App not allowed to call SPO with user\_impersonation" (Hinweis auf Graph als Lösung)<span class="" data-state="closed" style="color: var(--vscode-foreground);"><span class="ms-1 inline-flex max-w-full items-center relative top-[-0.094rem] animate-[show_150ms_ease-in]" data-testid="webpage-citation-pill"><a href="https://stackoverflow.com/questions/76629048/accessing-sharepoint-via-visual-studio-app-is-not-allowed-to-call-spo-with-user#:~:text=,error" target="_blank" rel="noopener" alt="https://stackoverflow.com/questions/76629048/accessing-sharepoint-via-visual-studio-app-is-not-allowed-to-call-spo-with-user#:~:text=,error" class="flex h-4.5 overflow-hidden rounded-xl px-2 text-[9px] font-medium text-token-text-secondary! bg-[#F4F4F4]! dark:bg-[#303030]! transition-colors duration-150 ease-in-out">stackoverflow.com</a></span></span>